In [1]:
class LexicalError(Exception):
    pass
class SemanticError(Exception):
    pass
class SyntaxError(Exception):
    pass

In [2]:
classes = {'L': 'abcdfghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ',
           'e':'e',
           'N': '1234567890',
           '+': '+-',
           '=': '=',
           '>': '>',
           '<': '<',
           '!': '!',
           'SS': '{}[]()/*,\n',
           '.':'.'}


def minus(class_name, lexem_table):
    if (class_name == 'N'):
        if (len(lexem_table) > 0 and lexem_table[-1][2] in [100, 101, 28]):
            return (False, True)
        else:
            return (3, True)
    elif (class_name == '.'):
        if (len(lexem_table) > 0 and lexem_table[-1][2] in [100, 101, 28]):
            return (False, True)
        else:
            return (8, True)
        
    elif(class_name == 'separator'):
        return (False, True)
        
    else:
        return (False, False)
    
    

automa = {
     1: ({'L': 2, 'N': 3, '+': 'minus_condition', '.': 8, '=': 15, '>': 12, '<': 9, '!': 18, 'SS': 20, 'e': 2}, False),
     2: ({'L': 2, 'N': 2, 'e': 2}, True, 'IDN'),
     3: ({'N': 3, 'e': 5, '.': 4}, True, 'CON'),
     4: ({'N': 4, 'e': 5}, True, 'CON'),
     5: ({'N': 6, '+': 7}, True, 'CON'),
     6: ({}, True, 'CON'),
     7: ({'N': 6}, False),
     8: ({'N': 4}, True, 'CON'),
     9: ({'<': 10, '=': 11}, True),
     10: ({}, True),
     11: ({}, True),
     12: ({'=': 14, '>': 13}, True),
     13: ({}, True),
     14: ({}, True),
     15: ({'=': 16}, True),
     16: ({}, True),
     17: ({}, True), 
     18: ({'=': 19}, False ), 
     19: ({}, True),
     20: ({}, True), 
     'minus_condition': minus
}


lexems_table = {'int': 1,
          'float': 2,
          'for': 3,
          'rof': 4,
          'while': 5,
          'do': 6,
          'print': 7,
          'input': 8,
          'by': 9,
          'if': 10,
          '\n': 12,
          ',': 13,
          '=': 14,
          '+': 15,
          '-': 16,
          '*': 17,
          '/': 18,
          '>': 19,
          '<': 20,
          '==': 21,
          '>=': 22,
          '<=': 23,
          '!=': 24,
          '[': 25,
          ']': 26,
          '(': 27,
          ')': 28,
          '{': 29,
          '}': 30,
          'to':31}

In [3]:
class LexicalAnalyser():
    def __init__(self, automa, classes, lexems_table, IDN, CON):
        self.automa = automa
        self.classes = classes
        self.lexems_table = lexems_table
        self.IDN_code = IDN
        self.CON_code = CON
        
        
        
    def analyse(self, code):
        state = 1
        table_of_lexems = []
        table_of_constants = []
        table_of_id = {}
        lexem = ''
        row = 1
        const_id = 0
        for char in code:
            class_name = self.get_class(char)
            if (class_name == 'separator' and state == 1):
                continue
            if(char == '\n'):
                row=row+1
    
            if(callable(self.automa[state])):
                state, is_finite = self.automa[state](class_name, table_of_lexems)
            else:
                current_state_info = self.automa[state]
                curves = self.automa[state][0]
                is_finite = self.automa[state][1]

                state = curves.get(class_name, False)
            
            if(state == False and is_finite == True):
                lexem_code = self.lexems_table.get(lexem, False)
                if (lexem_code == False):
                    if (current_state_info[2] == 'IDN'):
                        
                        if(table_of_lexems[-1][1] in ['int', 'float']):
                                if(lexem in table_of_id):
                                    raise SyntaxError('Variable {} has already been declared'.format(char))
                                   
                                lexem_code = self.IDN_code                                
                                table_of_id[lexem] = table_of_lexems[-1][1]
                                table_of_lexems.append([row, lexem, self.IDN_code, True, False])
                                
                        elif(lexem in table_of_id):
                            table_of_lexems.append([row, lexem, self.IDN_code, True, False])
                            
                        else:                                
                            print(table_of_lexems[-1][1])
                            raise SyntaxError('Undeclared variable {} is used at {} row'.format(char, row))
                        
                    else:
                        lexem_code = self.CON_code
                        table_of_constants.append([const_id, lexem, self.get_CON_type(lexem)])
                        table_of_lexems.append([row, lexem, self.CON_code, False, const_id])
                        const_id += 1
                else:
                    table_of_lexems.append([row, lexem, lexem_code, False, False])  #выделили лексему
                
                state = 1
                
                if(class_name == 'separator'):
                    lexem = ''
                    continue 
                    
                if(callable(self.automa[state])):
                    state, is_finite = self.automa[state](class_name, table_of_lexems)
                else: 
                    curves = self.automa[state][0]
                    state = curves.get(class_name, False)
                    
                if(state == False):
                    raise LexicalError("Lexical error {} at row  {}".format(char, row))
                lexem = char
            elif(state == False and is_finite == False):
                raise LexicalError("Lexical error {} at row  {}".format(char, row))
            else:
                lexem += char
        return table_of_lexems, table_of_constants, table_of_id

        
        
    def get_class(self, char):
        if (char in ' \t'):
            return 'separator'
        for key, value in self.classes.items():
            if (char in value):  
                return key
        raise LexicalError("Character '{}' does not belong to any of the classes at {} row".format(char, row) )
        
    def get_CON_type(self, lexem):
        try:
            int(lexem)
            return 'int'
        except ValueError:
            return 'float' 


In [4]:
def returning_lextb(w1):
    return w1, len(w1), 5
def returning_CONtb(w2):
    return w2, len(w2), 3
def returning_IDtb(w3):
    return w3, len(w3), 2

In [18]:
class SyntaxAnalyser():
    def __init__(self):
        self.i = 0
        
        
    def isProg(self, lexems):
        try:
            if(self.isSpOgolosh(lexems)):
                if(lexems[self.i][2] == 12):
                    self.i+=1
                    if(lexems[self.i][2] == 12):
                        self.i+=1
                  
                
                
                        if(self.isSpOperat(lexems)):
                            pass
                        else:
                            raise SyntaxError('No list of operators at row {}'.format(lexems[self.i][0]))
                    else:
                        raise SyntaxError('Missing Enter at row {}'.format(lexems[self.i][0]))
                else:
                    print(lexems[self.i])
                    raise SyntaxError('Missing Enter  at row {}'.format(lexems[self.i][0]))
            else:
                raise SyntaxError('No listing of ad variables at row {}'.format(lexems[self.i][0]))
        except IndexError as err:
            raise SyntaxError('incomplit command at row {}'.format(lexems[self.i-1][0]))
            
    def isSpOgolosh(self, lexems):
        if(self.isOgolosh(lexems)):
            while(lexems[self.i][2] == 12):
                self.i+=1
                if(lexems[self.i][2] == 12):
                    self.i-=1
                    return True
                if(self.isOgolosh(lexems)):                    
                    continue
                else:
                    raise SyntaxError('Incorrect variable declaration at row {}'.format(lexems[self.i][0]))
            return True
        else:
            return False
        
        
    def isOgolosh(self, lexems):
        if(self.isType(lexems)):
            if(self.isPrisv(lexems)):
                return True
            else:
                raise SyntaxError('Invalid assignment syntax at row {}'.format(lexems[self.i][0]))
        else:
            return False
        
        
    def isType(self, lexems):
        if(lexems[self.i][2] == 1 or lexems[self.i][2] == 2 ):
            self.i+=1
            return True
        else:
            return False
        
        
    def isPrisv(self, lexems):
        if(self.isId(lexems)):
            if(lexems[self.i][2] == 14):
                self.i+=1
                
                if(self.isE(lexems)):
                    return True
                else:
                    raise SyntaxError('Incorrect variable value at row {}'.format(lexems[self.i][0]))
            else:
                raise SyntaxError('Missing sign = at row {}'.format(lexems[self.i][0]))
        else:
            return False
        
        
    def isId(self, lexems):
        if(lexems[self.i][2] == 100):
            self.i+=1
            return True
        else:
            return False
        
        
        
    def isSpOperat(self, lexems):
        if(self.isOperat(lexems)):
            if(lexems[self.i][2] == 12):
                self.i+=1
                while(self.i < len(lexems)):                    
                    if(self.isOperat(lexems)):
                        if(lexems[self.i][2] == 12):
                            self.i+=1
                            continue
                        else:
                            raise SyntaxError('Missing Enter at row {}'.format(lexems[self.i][0])) 
                    else:
                        print(lexems[self.i])
                        raise SyntaxError('Operator is incorrectly specified at row {}'.format(lexems[self.i][0])) 
                return True        
            else:
                print(lexems[self.i])
                raise SyntaxError('Missing Enter at row {}'.format(lexems[self.i][0]))
        else:
            return False
        
    def isOperat(self, lexems):
        if(self.isCycle(lexems) or self.isUmovnPerehid(lexems) or 
           self.isPrisv(lexems) or self.isVvid(lexems) or self.isVuvid(lexems)):
            return True
        else:
            return False
        
        
    def isCycle(self, lexems):
        if(lexems[self.i][2] == 3):
            self.i+=1
            if(self.isId(lexems)):
                if(lexems[self.i][2] == 14):
                    self.i+=1
                    if(self.isE(lexems)):
                        if(lexems[self.i][2] == 31):
                            self.i+=1
                            if(self.isE(lexems)):
                                if(lexems[self.i][2] == 9):
                                    self.i+=1
                                    if(self.isE(lexems)):
                                        if(lexems[self.i][2] == 5):
                                            self.i+=1
                                            if(lexems[self.i][2] == 27):
                                                self.i+=1
                                                if(self.isVidnosh(lexems)):
                                                    if(lexems[self.i][2] == 28):
                                                        self.i+=1
                                                        if(self.isSpOperatCycle(lexems)):
                                                            if(lexems[self.i][2] == 4):
                                                                self.i+=1
                                                                return True
                                                            else:
                                                                raise SyntaxError('Error in the word ROF at row {}'.format(lexems[self.i][0]))
                                                        else:
                                                            raise SyntaxError('Incorrect statement list at row {}'.format(lexems[self.i][0]))
                                                    else:
                                                        raise SyntaxError('Missing character ) at row {}'.format(lexems[self.i][0]))
                                                else:
                                                    raise SyntaxError('Incorrect comparison at row {}'.format(lexems[self.i][0]))
                                            else:
                                                raise SyntaxError('Missing character ( at row {}'.format(lexems[self.i][0]))       
                                        else:
                                            raise SyntaxError('Error in the word WHILE at row {}'.format(lexems[self.i][0]))                         
                                    else:
                                        raise SyntaxError('Incorrect variable value at row {}'.format(lexems[self.i][0]))
                                else:
                                    raise SyntaxError('Error in the word BY  at row {}'.format(lexems[self.i][0]))
                            else:
                                raise SyntaxError('Incorrect variable value at row {}'.format(lexems[self.i][0]))
                        else:
                            raise SyntaxError('Error in the word TO at row {}'.format(lexems[self.i][0])) 
                    else:
                        raise SyntaxError('Incorrect variable value at row {}'.format(lexems[self.i][0]))
                else:
                    raise SyntaxError('Missing sign = at row {}'.format(lexems[self.i][0]))
            else:
                raise SyntaxError('Incorrect identifier (variable) at row {}'.format(lexems[self.i][0]))        
        else:
            return False
        
    def isUmovnPerehid(self, lexems):
        if(lexems[self.i][2] == 10):
            self.i+=1
            if(self.isVidnosh(lexems)):
                if(lexems[self.i][2] == 29):
                    self.i+=1
                    if(self.isSpOperatUmovnPerehid(lexems)):
                        if(lexems[self.i][2] == 30):
                            self.i+=1
                            return True
                        else:
                            raise SyntaxError('Missing character }} at row {}'.format(lexems[self.i][0]))
                    else:
                        print(lexems[self.i])
                        raise SyntaxError('Incorrect statement list at row {}'.format(lexems[self.i][0]))
                else:
                    raise SyntaxError('Missing character {{ at row {}'.format(lexems[self.i][0]))
            else:
                raise SyntaxError('Incorrect comparison at row {}'.format(lexems[self.i][0]))
        else:
            return False
        
        
    def isVvid(self, lexems):
        if(lexems[self.i][2] == 8):
            self.i+=1
            if(self.isId(lexems)):
                while(lexems[self.i][2] == 13):
                    self.i+=1
                    if(isId()):
                        continue
                    else:
                        raise SyntaxError('Incorrect identifier (variable) at row {}'.format(lexems[self.i][0]))                        
                return True
            else:
                raise SyntaxError('Incorrect identifier (variable) at row {}'.format(lexems[self.i][0]))
        else:
            return False
        
        
    def isVuvid(self, lexems):
        if(lexems[self.i][2] == 7):
            self.i+=1
            if(self.isId(lexems)):
                while(lexems[self.i][2] == 13):
                    self.i+=1
                    if(isId()):
                        continue
                    else:
                        raise SyntaxError('Incorrect identifier (variable) at row {}'.format(lexems[self.i][0]))                        
                return True
            else:
                raise SyntaxError('Incorrect identifier (variable) at row {}'.format(lexems[self.i][0]))
        else:
            return False
        
        
    def isVidnosh(self, lexems):
        if(self.isE(lexems)):
            if(self.isPorivn(lexems)):
                if(self.isE(lexems)):
                    return True
                else:
                    print(lexems[self.i])
                    raise SyntaxError('Incorrect variable value at row {}'.format(lexems[self.i][0]))
            else:
                raise SyntaxError('Incorrect character of comparison at row {}'.format(lexems[self.i][0]))
        else:
            return False
        
        
    def isPorivn(self, lexems):
        if(lexems[self.i][2] == 19 or lexems[self.i][2] ==20 or lexems[self.i][2] == 21 or 
          lexems[self.i][2] == 22 or lexems[self.i][2] == 23 or lexems[self.i][2] ==24):
            self.i+=1
            return True
        else:
            return False
              
        
    def isE(self, lexems):
        if(self.isT(lexems)):
            while(lexems[self.i][2] == 15 or lexems[self.i][2] == 16):
                self.i+=1
                if(self.isT(lexems)):
                    continue
                else:
                    raise SyntaxError('Incorrect identifier (variable) at row {}'.format(lexems[self.i][0]))
            return True
        else:
            return False
        
        
    def isT(self, lexems):
        if(self.isF(lexems)):
            while(lexems[self.i][2] == 17 or lexems[self.i][2] == 18):
                self.i+=1
                if(self.isF(lexems)):
                    continue
                else:
                    raise SyntaxError('Incorrect identifier (variable) at row {}'.format(lexems[self.i][0]))
            return True
        else:
            return False
            
        

    def isF(self, lexems):
        if(self.isId(lexems)):
            return True
        else:
            if(lexems[self.i][2] == 27):
                self.i+=1
                if(self.isE(lexems)):
                    if(lexems[self.i][2] == 28):
                        self.i+=1
                        return True
                    else:
                        raise SyntaxError('Missing ) at row {}'.format(lexems[self.i][0]))
                else:
                    raise SyntaxError('Variable error at row {}'.format(lexems[self.i][0]))
            else:
                if(self.isKPT(lexems)):
                    return True
                else:
                    return False
        
        
    def isKPT(self, lexems):
        if(lexems[self.i][2] == 101):
            self.i+=1
            return True
        else:
            return False
        
    def isSpOperatCycle(self, lexems):
        if(self.isOperat(lexems)):
            if(lexems[self.i][2] == 12):
                self.i+=1
                while(lexems[self.i][2] != 4):                    
                    if(selfs.isOperat(lexems)):
                        if(lexems[self.i][2] == 12):
                            self.i+=1
                            continue
                        else:
                            raise SyntaxError('Missing Enter at row {}'.format(lexems[self.i][0])) 
                    else:
                        raise SyntaxError('Operator is incorrectly specified at row {}'.format(lexems[self.i][0])) 
                return True        
            else:
                print(lexems[self.i])
                raise SyntaxError('Missing Enter at row {}'.format(lexems[self.i][0]))
        else:
            return False
        
    def isSpOperatUmovnPerehid(self, lexems):
        if(self.isOperat(lexems)):
            if(lexems[self.i][2] == 12):
                self.i+=1
                while(lexems[self.i][2] != 30):   
                    if(self.isOperat(lexems)):
                        if(lexems[self.i][2] == 12):
                            self.i+=1
                            continue
                        else:
                            raise SyntaxError('Missing Enter at row {}'.format(lexems[self.i][0])) 
                    else:
                        raise SyntaxError('Operator is incorrectly specified at row {}'.format(lexems[self.i][0])) 
                
                return True        
            else:
                print(lexems[self.i])
                raise SyntaxError('Missing Enter at row {}'.format(lexems[self.i][0]))
        else:
            return False

In [ ]:
from tkinter import *
from tkinter import ttk
from tkinter import filedialog
import tkinter as tk
from tkinter import messagebox


def Quit(ev):
    global root
    root.destroy()
    
def LoadFile(ev): 
    fn = filedialog.Open(root, filetypes = [('*.txt files', '.txt')]).show()
    if fn == '':
        return
    textbox.delete('1.0', 'end') 
    textbox.insert('1.0', open(fn, 'rt').read())
    
def SaveFile(ev):
    fn = filedialog.SaveAs(root, filetypes = [('*.txt files', '.txt')]).show()
    if fn == '':
        return
    if not fn.endswith(".txt"):
        fn+=".txt"
    open(fn, 'wt').write(textbox.get('1.0', 'end'))
    
    
    
    
class SimpleTable(tk.Frame):
    def __init__(self, parent, rows=10, columns=2):
        
        tk.Frame.__init__(self, parent, background="black")
        self._widgets = []
        for row in range(rows):
            current_row = []
            for column in range(columns):
                label = tk.Label(self, text="%s/%s" % (row, column), 
                                 borderwidth=0, width=10)
                label.grid(row=row, column=column, sticky="nsew", padx=1, pady=1)
                current_row.append(label)
            self._widgets.append(current_row)

        for column in range(columns):
            self.grid_columnconfigure(column, weight=1)


    def set(self, row, column, value):
        widget = self._widgets[row][column]
        widget.configure(text=value)
        

        

def LexemTables(ev):
    window1 = tk.Toplevel(root)
    table, rows, columns = returning_lextb(w1)
    scrollbar1 = Scrollbar(window1)
    

    
    t = SimpleTable(window1, rows, columns)
    t.pack(side="top", fill="x")
    
    for row in range(0, rows):
        for column in range(columns):
            t.set(row, column, table[row][column])


def ConstTable(ev):
    window2 = tk.Toplevel(root)
    table, rows, columns = returning_CONtb(w2)   
     
    t = SimpleTable(window2, rows, columns)
    t.pack(side="top", fill="x")
    
    for row in range(0, rows):
        for column in range(columns):
            t.set(row, column, table[row][column])
            

def IDTable(ev):
    window3 = tk.Toplevel(root)
    table, rows, columns = returning_IDtb(w3)   
     
    t = SimpleTable(window3, rows, columns)
    t.pack(side="top", fill="x")
    
    i = 0
    for ident in table:
        t.set(i, 0, ident)
        t.set(i, 1, table[ident])
        i += 1
            
def Process(ev):
    if textbox.compare("end-1c", "==", "1.0"):
        print("the widget is empty")
    else:
        global code 
        code = textbox.get("1.0",END) 
        global w1, w2, w3
        try:
            w1, w2, w3 = LexicalAnalyser(automa, classes, lexems_table, 100, 101 ).analyse(code)
            syntaxanalyser = SyntaxAnalyser()
            syntaxanalyser.isProg(w1)
        except LexicalError as err:
            messagebox.showerror("Lexical Error", str(err))
        except SyntaxError as err:
            messagebox.showerror("Syntax Error", str(err))
        except SemanticError as err:
            messagebox.showerror("Semantic Error", str(err))
        except IndexError as err:
            messagebox.showerror("Syntax Error", str(err))
            
        
root = Tk()
root.configure(background='black')
panelFrame = Frame(root, height = 60, bg = 'gray')
textFrame = Frame(root, height = 340, width = 600)

panelFrame.pack(side = 'top', fill = 'x')
textFrame.pack( fill = 'both', expand = 1)


textbox = Text(textFrame, font='Arial 14', wrap='word')
scrollbar = Scrollbar(textFrame)
# scrollbar1 = Scrollbar(errorFrame)



scrollbar['command'] = textbox.yview
textbox['yscrollcommand'] = scrollbar.set

# scrollbar1['command'] = errorbox.yview
# errorbox['yscrollcommand'] = scrollbar1.set

textbox.pack(side = 'left', fill = 'both', expand = 1)
scrollbar.pack(side = 'right', fill = 'y')

loadBtn = Button(panelFrame, text = 'Load')
saveBtn = Button(panelFrame, text = 'Save')
quitBtn = Button(panelFrame, text = 'Quit')
dispBtn1 = Button(panelFrame, text = 'Lexem table')
dispBtn2 = Button(panelFrame, text = 'CON table')
dispBtn3 = Button(panelFrame, text = 'ID table')
process = Button(panelFrame, text = 'Run', bg = 'red')

loadBtn.bind("<Button-1>", LoadFile)
saveBtn.bind("<Button-1>", SaveFile)
quitBtn.bind("<Button-1>", Quit)
dispBtn1.bind("<Button-1>", LexemTables)
dispBtn2.bind("<Button-1>", ConstTable)
dispBtn3.bind("<Button-1>", IDTable)
process.bind("<Button-1>", Process)

loadBtn.place(x = 10, y = 10, width = 40, height = 40)
saveBtn.place(x = 60, y = 10, width = 40, height = 40)
quitBtn.place(x = 110, y = 10, width = 40, height = 40)
dispBtn1.place(x = 160, y = 10, width = 95, height = 40)
dispBtn2.place(x = 260, y = 10, width = 80, height = 40)
dispBtn3.place(x = 350, y = 10, width = 80, height = 40)
process.place(x = 900, y = 10, width = 40, height = 40)

root.mainloop()

vidnosh:  [4, 'a', 100, True, False]
vidnosh:  [6, 'a', 100, True, False]
vidnosh:  [4, 'a', 100, True, False]
vidnosh:  [7, '\n', 12, False, False]
vidnosh:  [5, 'a', 100, True, False]
vidnosh:  [7, 'a', 100, True, False]
vidnosh:  [5, 'a', 100, True, False]
vidnosh:  [7, 'a', 100, True, False]
vidnosh:  [5, 'a', 100, True, False]
vidnosh:  [7, 'a', 100, True, False]
vidnosh:  [5, 'a', 100, True, False]
vidnosh:  [7, 'a', 100, True, False]
vidnosh:  [5, 'a', 100, True, False]
vidnosh:  [7, 'a', 100, True, False]
